## Exercise 6-1

# Entity-MLP 

In this exercise we will discuss relational model based on multi layer perceptron(MLP). We will create composite representation of triple and will use non linear function to score the triple for link prediction.
Tensor based methods explicitly models all pairwise computation thus are not scalable on knowledge graph with large number of relations. In MLP based model instead of considering all interactions of latent features interactions are learned via $W_k$.


In the following we define E-MLP to model triples in a given knowledge graph $KG=<e_i,r_k, e_j>$:

$f_{ijk}^{E-MLP} = r_{k}^Tg(h_{ijk})$

$h_{ijk} = W_k^Tx_{ij}^{E-MLP}$

$x_{ij}^{E-MLP} = [a_i,a_j]$

where g is a non linearity defined as element-wise operation $g(v_i)=tanh(v_i)$ over hidden representations $h_{ijk}$. Every element of hidden representation i.e. $h_{ijk}$ is obtained by adding different weight component of the composite entity representation $x_{ij}$. Each element in composite representation is obtained by stacking pairwise latent representation of head and tail $[a_i, a_j]$.

$W_{k}$ is the hidden representation defined for each relation. 


Each element of $f_{ijk}^{E-MLP}$ is a confidence of triple $<e_i,r_k, e_j>$. Similar to previous exercise we can formulate a link prediction problem using binary cross entropy loss function and solve it using gradient based methods:

$L_{ijk} = x_{ijk} log \sigma (f_{ijk}^{E-MLP}) + (1-x_{ijk}) log \sigma (f_{ijk}^{E-MLP})$

where $x_{ijk}=1$ if triple $<e_i,r_k,e_j>$ exists and $x_{ijk} = 0$ otherwise.

For evaluations of method we will use the kinship dataset representing 26 relations (brother, sister, father,...} between 104 people. 

In [123]:
import torch.nn as nn
import torch
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable
from utils import get_minibatches, sample_negatives, accuracy, auc
from time import time

In [155]:
class EMLP(nn.Module):
    """
    ER-MLP: Entity MLP
    ---------------------------
    """

    def __init__(self, n_e, n_r, k, lam, gpu=False):
        """
        E-MLP: Entity-MLP
        ---------------------------

        Params:
        -------
            n_e: int
                Number of entities in dataset.

            n_r: int
                Number of relationships in dataset.

            k: int
                Embedding size.

            lam: float
                Prior strength of the embeddings. Used to constaint the
                embedding norms inside a (euclidean) unit ball. The prior is
                Gaussian, this param is the precision.

            gpu: bool, default: False
                Whether to use GPU or not.
        """
        super(EMLP, self).__init__()

        # Hyperparams
        self.n_e = n_e
        self.n_r = n_r
        self.k = k
        self.lam = lam
        self.gpu = gpu
        # Nets
        self.emb_E = nn.Embedding(self.n_e, self.k)
        #self.emb_R = nn.Embedding(self.n_r, self.k*2)
        self.emb_R = []
        for i in range(n_r):
            self.emb_R.append(nn.Sequential(nn.Linear(self.k*2, self.k),nn.ReLU(),))
        
        self.mlp = nn.Sequential(
            nn.Linear(self.k, 1),
        )

        self.embeddings = [self.emb_E]
        self.initialize_embeddings()

        # Xavier init
        for layer in self.emb_R:
            for p in layer.modules():
                if isinstance(p, nn.Linear):
                    in_dim = p.weight.size(0)
                    p.weight.data.normal_(0, 1/np.sqrt(in_dim/2))
        for p in self.mlp.modules():
            if isinstance(p, nn.Linear):
                in_dim = p.weight.size(0)
                p.weight.data.normal_(0, 1/np.sqrt(in_dim/2))

        # Copy all params to GPU if specified
        if self.gpu:
            self.cuda()

    def forward(self, X):
        # Decompose X into head, relationship, tail
        hs, ls, ts = X[:, 0], X[:,1] , X[:, 2]

        if self.gpu:
            hs = Variable(torch.from_numpy(hs).cuda())
            ts = Variable(torch.from_numpy(ts).cuda())
            #ls = Variable(torch.from_numpy(ls).cuda())            
        else:
            hs = Variable(torch.from_numpy(hs))
            ts = Variable(torch.from_numpy(ts))
            #ls = Variable(torch.from_numpy(ls))            

        # Project to embedding, each is M x k
        e_hs = self.emb_E(hs)
        e_ts = self.emb_E(ts)
        # Forward
        phi = torch.cat([e_hs, e_ts], 1)  # M x 2k
        x = Variable(torch.zeros(len(ls),k))
        for i,rel in enumerate(ls):
            x[i] = self.emb_R[rel](phi[i])
        y = self.mlp(x)
        return y.view(-1, 1)
    
    def predict(self, X, sigmoid=False):
        """
        Predict the score of test batch.

        Params:
        -------
        X: int matrix of M x 3, where M is the (mini)batch size
            First row contains index of head entities.
            Second row contains index of relationships.
            Third row contains index of tail entities.

        sigmoid: bool, default: False
            Whether to apply sigmoid at the prediction or not. Useful if the
            predicted result is scores/logits.

        Returns:
        --------
        y_pred: np.array of Mx1
        """
        y_pred = self.forward(X).view(-1, 1)

        if sigmoid:
            y_pred = F.sigmoid(y_pred)

        if self.gpu:
            return y_pred.cpu().data.numpy()
        else:
            return y_pred.data.numpy()
        
    def log_loss(self, y_pred, y_true, average=True):
        """
        Compute log loss (Bernoulli NLL).

        Params:
        -------
        y_pred: vector of size Mx1
            Contains prediction logits.

        y_true: np.array of size Mx1 (binary)
            Contains the true labels.

        average: bool, default: True
            Whether to average the loss or just summing it.

        Returns:
        --------
        loss: float
        """
        if self.gpu:
            y_true = Variable(torch.from_numpy(y_true.astype(np.float32)).cuda())
        else:
            y_true = Variable(torch.from_numpy(y_true.astype(np.float32)))

        nll = F.binary_cross_entropy_with_logits(y_pred, y_true, size_average=average)

        norm_E = torch.norm(self.emb_E.weight, 2, 1)

        # Penalize when embeddings norms larger than one
        nlp1 = torch.sum(torch.clamp(norm_E - 1, min=0))

        if average:
            nlp1 /= nlp1.size(0)

        return nll + self.lam*nlp1

    def ranking_loss(self, y_pos, y_neg, margin=1, C=1, average=True):
        """
        Compute loss max margin ranking loss.

        Params:
        -------
        y_pos: vector of size Mx1
            Contains scores for positive samples.

        y_neg: np.array of size Mx1 (binary)
            Contains the true labels.

        margin: float, default: 1
            Margin used for the loss.

        C: int, default: 1
            Number of negative samples per positive sample.

        average: bool, default: True
            Whether to average the loss or just summing it.

        Returns:
        --------
        loss: float
        """
        M = y_pos.size(0)

        y_pos = y_pos.view(-1).repeat(C)  # repeat to match y_neg
        y_neg = y_neg.view(-1)

        # target = [-1, -1, ..., -1], i.e. y_neg should be higher than y_pos
        target = -np.ones(M*C, dtype=np.float32)

        if self.gpu:
            target = Variable(torch.from_numpy(target).cuda())
        else:
            target = Variable(torch.from_numpy(target))

        loss = F.margin_ranking_loss(
            y_pos, y_neg, target, margin=margin, size_average=average
        )

        return loss

    def normalize_embeddings(self):
        for e in self.embeddings:
            e.weight.data.renorm_(p=2, dim=0, maxnorm=1)

    def initialize_embeddings(self):
        r = 6/np.sqrt(self.k)

        for e in self.embeddings:
            e.weight.data.uniform_(-r, r)

        self.normalize_embeddings()
        

In [156]:
# Set random seed
randseed = 9999
np.random.seed(randseed)
torch.manual_seed(randseed)

In [157]:
# Data Loading
# Load dictionary lookups
idx2ent = np.load('data/kinship/bin/idx2ent.npy')
idx2rel = np.load('data/kinship/bin/idx2rel.npy')

n_e = len(idx2ent)
n_r = len(idx2rel)

# Load dataset
X_train = np.load('data/kinship/bin/train.npy')
X_val = np.load('data/kinship/bin/val.npy')
y_val = np.load('data/kinship/bin/y_val.npy')

X_val_pos = X_val[y_val.ravel() == 1, :]  # Take only positive samples

M_train = X_train.shape[0]
M_val = X_val.shape[0]

# Model Parameters
k = 50
embeddings_lambda = 0
model = EMLP(n_e=n_e, n_r=n_r, k=k, lam=embeddings_lambda, gpu= False)

In [158]:
normalize_embed = True
C = 10 # Negative Samples
n_epoch = 20
lr = 0.1
lr_decay_every = 20
#weight_decay = 1e-4
mb_size = 100  
print_every = 100
average = True
# Optimizer Initialization
#solver = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
solver = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# Begin training
for epoch in range(n_epoch):
    print('Epoch-{}'.format(epoch+1))
    print('----------------')
    it = 0
    # Shuffle and chunk data into minibatches
    mb_iter = get_minibatches(X_train, mb_size, shuffle=True)

    # Anneal learning rate
    lr = lr * (0.5 ** (epoch // lr_decay_every))
    for param_group in solver.param_groups:
        param_group['lr'] = lr

    for X_mb in mb_iter:
        start = time()

        # Build batch with negative sampling
        m = X_mb.shape[0]
        # C x M negative samples
        X_neg_mb = sample_negatives(X_mb, n_e)
        X_train_mb = np.vstack([X_mb, X_neg_mb])

        y_true_mb = np.vstack([np.ones([m, 1]), np.zeros([m, 1])])

        # Training step
        y = model.forward(X_train_mb)
        loss = model.log_loss(y, y_true_mb, average=average)
        
        loss.backward()
        solver.step()
        solver.zero_grad()
        if normalize_embed:
            model.normalize_embeddings()

        end = time()
        # Training logs
        if it % print_every == 0:
            # Training auc
            pred = model.predict(X_train_mb, sigmoid=True)
            train_acc = auc(pred, y_true_mb)
            
            # Per class accuracy
            # pos_acc = accuracy(pred[:m], y_true_mb[:m])
            # neg_acc = accuracy(pred[m:], y_true_mb[m:])

            # Validation auc
            y_pred_val = model.forward(X_val)
            y_prob_val = F.sigmoid(y_pred_val)
            
            val_acc = auc(y_prob_val.data.numpy(), y_val)
            # Validation loss
            val_loss = model.log_loss(y_pred_val, y_val, average)

            print('Iter-{}; loss: {:.4f}; train_auc: {:.4f}; val_auc: {:.4f}; val_loss: {:.4f}; time per batch: {:.2f}s'
                    .format(it, loss.data[0], train_acc, val_acc, val_loss.data[0], end-start))

        it += 1

Epoch-1
----------------
Iter-0; loss: 1.2087; train_auc: 0.5060; val_auc: 0.4903; val_loss: 1.1569; time per batch: 0.07s
Epoch-2
----------------
Iter-0; loss: 0.7122; train_auc: 0.5897; val_auc: 0.5613; val_loss: 0.7441; time per batch: 0.08s
Epoch-3
----------------
Iter-0; loss: 0.7020; train_auc: 0.5817; val_auc: 0.6106; val_loss: 0.6894; time per batch: 0.08s
Epoch-4
----------------
Iter-0; loss: 0.6026; train_auc: 0.7365; val_auc: 0.6511; val_loss: 0.6602; time per batch: 0.07s
Epoch-5
----------------
Iter-0; loss: 0.6695; train_auc: 0.6401; val_auc: 0.6817; val_loss: 0.6450; time per batch: 0.08s
Epoch-6
----------------
Iter-0; loss: 0.6020; train_auc: 0.7586; val_auc: 0.6999; val_loss: 0.6330; time per batch: 0.08s
Epoch-7
----------------
Iter-0; loss: 0.6329; train_auc: 0.7089; val_auc: 0.7174; val_loss: 0.6198; time per batch: 0.07s
Epoch-8
----------------
Iter-0; loss: 0.6275; train_auc: 0.7168; val_auc: 0.7408; val_loss: 0.6046; time per batch: 0.08s
Epoch-9
--------

## Task: Experiment with different loss function

In above implementation we use binary cross entropy as a loss function. In following part repeat above experiments using ranking loss. 

In [122]:
###########################
###### Your Code Here
###########################